In [1]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn import cross_validation
from sklearn.preprocessing import Imputer

/home/m/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
data=pd.read_csv("./data_copy.csv",encoding="gbk")

In [3]:
miss=data.isnull().sum()
miss_rate=data.isnull().sum()/data.shape[0]
pd.concat([miss,miss_rate],axis=1,keys=['缺失值','缺失率'])

,缺失值,缺失率
Unnamed: 0,0,0.000000
custid,0,0.000000
trade_no,0,0.000000
bank_card_no,0,0.000000
low_volume_percent,2,0.000421
middle_volume_percent,2,0.000421
take_amount_in_later_12_month_highest,0,0.000000
trans_amount_increase_rate_lately,3,0.000631
trans_activity_month,2,0.000421
trans_activity_day,2,0.000421


In [4]:
#查看object型变量取值分布情况
obj_col = (data.dtypes == 'object')[(data.dtypes == 'object') == True].index.tolist()
for col in obj_col:
    print (data[col].value_counts())
    print ('\n')

卡号1    4754
Name: bank_card_no, dtype: int64


一线城市    3403
三线城市    1064
境外       150
二线城市     131
其他城市       4
Name: reg_preference_for_trad, dtype: int64


xs    4754
Name: source, dtype: int64


李明     5
李杰     5
张龙     5
李波     4
张伟     4
刘强     4
陈鹏     4
张超     4
陈波     4
刘凯     3
王强     3
张杰     3
杨浩     3
王毅     3
张健     3
李鹏     3
李博     3
杨明     3
张平     3
刘磊     3
张旭     3
王剑     3
张强     3
李栋     3
陈洁     3
王帅     3
张锋     2
杨建     2
马强     2
何涛     2
      ..
周骆斌    1
谢光宁    1
余豪     1
曹刚     1
高针净    1
牛国斌    1
刘海滨    1
郭超     1
杨军昆    1
陈奎宇    1
余少平    1
沙海军    1
潘兴利    1
黄永志    1
周盼盼    1
刘京超    1
石杰     1
杨泉杰    1
张康康    1
韩宁     1
李健华    1
徐厦     1
姚心空    1
李洪霞    1
郑华锋    1
周涛     1
文三波    1
纪中友    1
付涛     1
陈建滨    1
Name: id_name, Length: 4309, dtype: int64


2018-04-14    423
2018-05-06    223
2018-05-05    214
2018-05-04    210
2018-05-02    204
2018-05-03    201
2018-04-13    177
2018-04-23    166
2018-04-28    139
2018-05-01    133
2018-04-26    132
2018-04-27 

In [5]:
#删除部分列
data=data.drop(['Unnamed: 0','custid','id_name','bank_card_no','source','student_feature','trade_no'], axis = 1)

In [6]:
#处理缺失值,用'其他城市'填充
data['reg_preference_for_trad'].fillna("其他城市",inplace=True)

#使用onehot编码
data = data.join(pd.get_dummies(data['reg_preference_for_trad']))
data.drop(['reg_preference_for_trad'],axis = 1,inplace = True)

In [7]:
#将时间序列object 修改为 时间序列
data['latest_query_time'] = pd.to_datetime(data['latest_query_time'], format='%Y-%m-%d %H:%M:%S.%f')
data['loans_latest_time'] = pd.to_datetime(data['loans_latest_time'], format='%Y-%m-%d %H:%M:%S.%f')

In [8]:
#将时间序列 修改为int
import datetime
a = (datetime.datetime.today() - data['loans_latest_time'])
b = map(lambda a:a.days, a)
data['loans_latest_time'] = pd.Series(b)

a = (datetime.datetime.today() - data['latest_query_time'])
b = map(lambda a:a.days, a)
data['latest_query_time'] = pd.Series(b)

In [9]:
#筛选无关特征，相关系数
corr=data.corr()
corr_1=np.abs(corr['status'])
corr_1.sort_values(ascending=False)

status                                     1.000000
trans_fail_top_count_enum_last_1_month     0.332256
history_fail_fee                           0.320033
loans_overdue_count                        0.279125
loans_score                                0.259092
latest_one_month_fail                      0.257269
apply_score                                0.244200
rank_trad_1_month                          0.147258
latest_one_month_suc                       0.138144
trans_fail_top_count_enum_last_6_month     0.133477
trans_fail_top_count_enum_last_12_month    0.117161
top_trans_count_last_1_month               0.117113
consfin_avg_limit                          0.091101
consfin_credit_limit                       0.083844
trans_day_last_12_month                    0.078892
avg_price_top_last_12_valid_month          0.071874
trans_top_time_last_1_month                0.068591
consume_top_time_last_1_month              0.066371
consfin_max_limit                          0.063307
latest_six_m

In [10]:
#用中值进行填充,致谢ruaruaxia思路
for i in data.keys():
    data[i].fillna(data[i].median(), inplace=True)
data.isnull().sum().sort_values(ascending=False)

境外                                            0
consume_mini_time_last_1_month                0
trans_top_time_last_1_month                   0
trans_top_time_last_6_month                   0
consume_top_time_last_1_month                 0
consume_top_time_last_6_month                 0
cross_consume_count_last_1_month              0
trans_fail_top_count_enum_last_1_month        0
trans_fail_top_count_enum_last_6_month        0
trans_fail_top_count_enum_last_12_month       0
max_cumulative_consume_later_1_month          0
avg_price_last_12_month                       0
max_consume_count_later_6_month               0
railway_consume_count_last_12_month           0
pawns_auctions_trusts_consume_last_1_month    0
pawns_auctions_trusts_consume_last_6_month    0
jewelry_consume_count_last_6_month            0
status                                        0
first_transaction_day                         0
trans_day_last_12_month                       0
avg_price_top_last_12_valid_month       

In [11]:
data.columns

Index(['low_volume_percent', 'middle_volume_percent',
       'take_amount_in_later_12_month_highest',
       'trans_amount_increase_rate_lately', 'trans_activity_month',
       'trans_activity_day', 'transd_mcc', 'trans_days_interval_filter',
       'trans_days_interval', 'regional_mobility', 'repayment_capability',
       'is_high_user', 'number_of_trans_from_2011', 'first_transaction_time',
       'historical_trans_amount', 'historical_trans_day', 'rank_trad_1_month',
       'trans_amount_3_month', 'avg_consume_less_12_valid_month', 'abs',
       'top_trans_count_last_1_month', 'avg_price_last_12_month',
       'avg_price_top_last_12_valid_month', 'trans_top_time_last_1_month',
       'trans_top_time_last_6_month', 'consume_top_time_last_1_month',
       'consume_top_time_last_6_month', 'cross_consume_count_last_1_month',
       'trans_fail_top_count_enum_last_1_month',
       'trans_fail_top_count_enum_last_6_month',
       'trans_fail_top_count_enum_last_12_month',
       'consume_

In [12]:
x_columns = []  # X的列属性(x变量)
for col in data.columns:
    if col not in ['latest_query_time','loans_latest_time','status']:
        x_columns.append(col)  
        
X = data[x_columns]  # 获取x变量
Y = data['status']   # 获取y标签(label)

In [15]:
# 使用随机会森林进行特征选择：
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier()
clf.fit(X, Y)

importance = clf.feature_importances_
indices = np.argsort(importance)[::-1]
features = X.columns
for f in range(X.shape[1]):
    print(("%2d) %-*s %f" % (f + 1, 30, features[f], importance[indices[f]])))

 1) low_volume_percent             0.048610
 2) middle_volume_percent          0.046491
 3) take_amount_in_later_12_month_highest 0.038195
 4) trans_amount_increase_rate_lately 0.029601
 5) trans_activity_month           0.027517
 6) trans_activity_day             0.025718
 7) transd_mcc                     0.019158
 8) trans_days_interval_filter     0.017214
 9) trans_days_interval            0.017162
10) regional_mobility              0.016868
11) repayment_capability           0.016781
12) is_high_user                   0.016284
13) number_of_trans_from_2011      0.016127
14) first_transaction_time         0.015596
15) historical_trans_amount        0.015268
16) historical_trans_day           0.014697
17) rank_trad_1_month              0.014591
18) trans_amount_3_month           0.014488
19) avg_consume_less_12_valid_month 0.014443
20) abs                            0.014439
21) top_trans_count_last_1_month   0.013972
22) avg_price_last_12_month        0.013828
23) avg_price_top_las

In [16]:
# 使用iv值进行特征选择(参考了群里大佬的代码)：
def calc_iv(df, feature, target, pr=False):
    """
    Set pr=True to enable printing of output.

    Output:
      * iv: float,
      * data: pandas.DataFrame
    """

    lst = []
    for i in range(df[feature].nunique()):  # nuinque()是查看该序列(axis=0/1对应着列或行)的不同值的数量个数
        val = list(df[feature].unique())[i]
        lst.append([feature,
                    val,  # Value
                    df[df[feature] == val].count()[feature],  # all
                    df[(df[feature] == val) & (df[target] == 0)].count()[feature],  # good rate
                    df[(df[feature] == val) & (df[target] == 1)].count()[feature]])  # bad rate
    data = pd.DataFrame(lst, columns=['Variable', 'Value', 'All', 'Good', 'Bad'])

    data['Share'] = data['All'] / data['All'].sum()
    data['Bad Rate'] = data['Bad'] / data['All']
    data['Distribution Good'] = (data['All'] - data['Bad']) / (data['All'].sum() - data['Bad'].sum())
    data['Distribution Bad'] = data['Bad'] / data['Bad'].sum()
    data['WoE'] = np.log(data['Distribution Good'] / data['Distribution Bad'])

    data = data.replace({'WoE': {np.inf: 0, -np.inf: 0}})

    data['IV'] = data['WoE'] * (data['Distribution Good'] - data['Distribution Bad'])

    data = data.sort_values(by=['Variable', 'Value'], ascending=[True, True])
    data.index = range(len(data.index))

    if pr:
        print(data)
        print("IV = ", data['IV'].sum())

    iv = data['IV'].sum()

    return iv, data

In [17]:
#循环输出每个特征的iv值
for x in x_columns:
    iv, datas = calc_iv(data, x, 'status')
    print('{}： {}'.format(x, iv))

/home/m/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:25: RuntimeWarning: divide by zero encountered in log


low_volume_percent： 0.03435233000814904
middle_volume_percent： 0.07145504224696433
take_amount_in_later_12_month_highest： 0.08329868129431747
trans_amount_increase_rate_lately： 0.2872982182102352
trans_activity_month： 0.07311662924513179
trans_activity_day： 0.3648952102643921
transd_mcc： 0.03609538449991616
trans_days_interval_filter： 0.09287874942209527
trans_days_interval： 0.09458570507641034
regional_mobility： 0.007947512041471004
repayment_capability： 0.4390959671989395
is_high_user： 0.005613886645064013
number_of_trans_from_2011： 0.05804149965599338
first_transaction_time： 0.4861222320898152
historical_trans_amount： 0.053932681011240866
historical_trans_day： 0.35232159217874237
rank_trad_1_month： 0.12815009257082632
trans_amount_3_month： 0.24484789224813222
avg_consume_less_12_valid_month： 0.013734163082283282
abs： 0.3727207019325916
top_trans_count_last_1_month： 0.07264620108656981
avg_price_last_12_month： 0.19973803729124645
avg_price_top_last_12_valid_month： 0.04462939978202485